In [76]:
import pandas as pd

In [77]:
df = pd.read_excel('../data_input/datasource.xlsx',sheet_name=0)

In [78]:
df_sample_plan = pd.read_excel('../data_input/Sampling Plan information.xlsx')

In [79]:
df_sample_plan.rename(columns={'Lot Size (from)':'Min','Lot Size (to)':'Max'},inplace=True)

In [80]:
df_sample_plan.loc[df_sample_plan.groupby(['Sampling Plan ID'])['Max'].idxmax(), 'Max'] = 99999999

In [81]:
df_normal = pd.read_excel('../data_input/datasource.xlsx',sheet_name=1)
df_reduce = pd.read_excel('../data_input/datasource.xlsx',sheet_name=2)

In [82]:
df_normal_Functional = df_normal.loc[df_normal['Criteria Type'] == 'Functional',['PIC','Sampling Plan ID']]
df_normal_Dimensional = df_normal.loc[df_normal['Criteria Type'] == 'Dimensional',['PIC','Sampling Plan ID']]
df_normal_Visual = df_normal.loc[df_normal['Criteria Type'] == 'Visual',['PIC','Sampling Plan ID']]

In [83]:
df_reduce_Functional = df_reduce.loc[df_reduce['Criteria Type'] == 'Functional',['PIC','Sampling Plan ID']]
df_reduce_Dimensional = df_reduce.loc[df_reduce['Criteria Type'] == 'Dimensional',['PIC','Sampling Plan ID']]
df_reduce_Visual = df_reduce.loc[df_reduce['Criteria Type'] == 'Visual',['PIC','Sampling Plan ID']]

In [84]:
df_reduce_Visual

,PIC,Sampling Plan ID
1,PIC-00001,188
2,PIC-00001,461
3,PIC-00001,189
4,PIC-00001,229
7,PIC-00002,189
...,...,...
653,Qual-99963,229
654,Qual-99963,241
655,Qual-99963,539
656,Qual-99963,267


In [85]:
def get_sample_size(row, df):
    # 在第三个数据框中查询符合条件的Sample Size
    sample_sizes = df[(df['Sampling Plan ID'] == row['Sampling Plan ID']) & 
                       (df['Min'] <= row['Qty EA']) & 
                       (df['Max'] >= row['Qty EA'])]['Sample Size']
    # 如果找到符合条件的Sample Size，则返回最大值
    return sample_sizes.max() if not sample_sizes.empty else None


In [98]:
# df_base = (
#     df
# .assign(**{"Inspection Date" : lambda d : pd.to_datetime(d['Inspection Date'])})
# .assign(key=lambda d: d.apply(lambda s:   
#                             s['ID'] if s['Path'] == 'QIM'   
#                             else '|'.join([str(s['Lot Number']),   
#                                            s['Inspection Date'].date().strftime('%Y-%m-%d'),   
#                                            str(s['Inspector']),   
#                                            str(s['Item Number'])]),   
#                           axis=1)
#         )
# .query('PIC != "Missing in ETQ"')
# .groupby(['key','PIC'],as_index=False)['Qty EA'].sum()
# )

In [96]:
df_base = (
    df
    .assign(key=lambda d: d.apply(lambda s: f"{s['Lot Number']}|{s['Inspection Date'].date()}|{s['Inspector']}|{s['Item Number']}" 
                                    if s['Path'] != 'QIM' 
                                    else s['ID'], axis=1))
    .query('PIC != "Missing in ETQ"')
    .groupby(['key', 'PIC'], as_index=False)['Qty EA'].sum()
)

In [94]:
df_base.to_excel('base.xlsx')

In [67]:
df_sample_size_normal_Functional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Functional,on='PIC',how='left'))
    .assign(**{"normal_Functional Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('normal_Functional Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Functional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\2639328354.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Functional.sum()


PIC                              PIC-00035PIC-00035PIC-00035PIC-00035PIC-00035P...
Qty EA                                                                6517335215.0
Sampling Plan ID                                                         1584147.0
normal_Functional Sample Size                                             202089.0
dtype: object

In [68]:
df_sample_size_normal_Dimensional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Dimensional,on='PIC',how='left'))
    .assign(**{"normal_Dimensional Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('normal_Dimensional Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Dimensional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\2333508455.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Dimensional.sum()


PIC                               PIC-00418PIC-00418PIC-00418PIC-00418PIC-00418P...
Qty EA                                                                 6517335215.0
Sampling Plan ID                                                          1440642.0
normal_Dimensional Sample Size                                             235002.0
dtype: object

In [69]:
df_sample_size_normal_Visual = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Visual,on='PIC',how='left'))
    .assign(**{"normal_Visual Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('normal_Visual Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Visual.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\36782525.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Visual.sum()


PIC                          PIC-00058PIC-00058PIC-00058PIC-00058PIC-00058P...
Qty EA                                                            6517335215.0
Sampling Plan ID                                                     1786566.0
normal_Visual Sample Size                                            1903730.0
dtype: object

In [70]:
df_sample_size_reduce_Functional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Functional,on='PIC',how='left'))
    .assign(**{"reduce_Functional Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('reduce_Functional Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Functional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\2526576968.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_reduce_Functional.sum()


PIC                              PIC-00035PIC-00035PIC-00035PIC-00035PIC-00035P...
Qty EA                                                                6517335215.0
Sampling Plan ID                                                         3847641.0
reduce_Functional Sample Size                                              79454.0
dtype: object

In [71]:
df_sample_size_reduce_Dimensional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Dimensional,on='PIC',how='left'))
    .assign(**{"reduce_Dimensional Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('reduce_Dimensional Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Dimensional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\2339830204.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_reduce_Dimensional.sum()


PIC                               PIC-00418PIC-00418PIC-00418PIC-00413PIC-00418P...
Qty EA                                                                 6517335215.0
Sampling Plan ID                                                          4312348.0
reduce_Dimensional Sample Size                                              91752.0
dtype: object

In [72]:
df_sample_size_reduce_Visual = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Visual,on='PIC',how='left'))
    .assign(**{"reduce_Visual Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('reduce_Visual Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Visual.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_12024\558336742.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_reduce_Visual.sum()


PIC                          PIC-00058PIC-00058PIC-00058PIC-00058PIC-00058P...
Qty EA                                                            6517335215.0
Sampling Plan ID                                                     4860663.0
reduce_Visual Sample Size                                             761486.0
dtype: object

In [73]:
df_sample_size_normal_Functional = df_sample_size_normal_Functional.iloc[:,[0,4]]
df_sample_size_normal_Dimensional = df_sample_size_normal_Dimensional.iloc[:,[0,4]]
df_sample_size_normal_Visual = df_sample_size_normal_Visual.iloc[:,[0,4]]
df_sample_size_reduce_Functional = df_sample_size_reduce_Functional.iloc[:,[0,4]]
df_sample_size_reduce_Dimensional = df_sample_size_reduce_Dimensional.iloc[:,[0,4]]
df_sample_size_reduce_Visual = df_sample_size_reduce_Visual.iloc[:,[0,4]]


In [ ]:
(
df_base
.merge(df_sample_size_normal_Functional,on='key',how='left')
.merge(df_sample_size_normal_Dimensional,on='key',how='left')
.merge(df_sample_size_normal_Visual,on='key',how='left')
.merge(df_sample_size_reduce_Functional,on='key',how='left')
.merge(df_sample_size_reduce_Dimensional,on='key',how='left')
.merge(df_sample_size_reduce_Visual,on='key',how='left')
.to_excel('./out_final.xlsx')
)

In [125]:
import pandas as pd
from functools import reduce

# 从datasource.xlsx读取指定的sheet
xls_datasource = pd.ExcelFile('../data_input/datasource.xlsx')
df = xls_datasource.parse(0)  # 基础数据DataFrame
df_normal = xls_datasource.parse(1)  # 正常检验标准DataFrame
df_reduce = xls_datasource.parse(2)  # 缩减检验标准DataFrame

# 从Sampling Plan information.xlsx读取特定的sheet
df_sample_plan = pd.read_excel('../data_input/Sampling Plan information.xlsx')

# 重命名列并调整Sampling Plan信息
df_sample_plan.rename(columns={'Lot Size (from)': 'Min', 'Lot Size (to)': 'Max'}, inplace=True)
df_sample_plan.loc[df_sample_plan.groupby(['Sampling Plan ID'])['Max'].idxmax(), 'Max'] = 99999999

# 简化DataFrame切片操作的函数
def get_criteria_df(criteria_df, criteria_type):
    return criteria_df[criteria_df['Criteria Type'] == criteria_type][['PIC', 'Sampling Plan ID']]

# 函数化重复逻辑
def create_sample_size_df(base_df, criteria_df, criteria_name):
    merged_df = pd.merge(base_df, criteria_df, on='PIC', how='left')
    merged_df[f"{criteria_name} Sample Size"] = merged_df.apply(lambda row: get_sample_size(row, df_sample_plan), axis=1)
    return merged_df.sort_values(by=f"{criteria_name} Sample Size", ascending=False).drop_duplicates('key')[['key', f"{criteria_name} Sample Size"]]

def get_sample_size(row, df_sample_plan):
    sample_sizes = df_sample_plan[(df_sample_plan['Sampling Plan ID'] == row['Sampling Plan ID']) & 
                                  (df_sample_plan['Min'] <= row['Qty EA']) & 
                                  (df_sample_plan['Max'] >= row['Qty EA'])]['Sample Size']
    return sample_sizes.max() if sample_sizes.notna().any() else None

# 处理基础DataFrame
df_base = (
    df
    .assign(Inspection_Date=lambda d: pd.to_datetime(d['Inspection Date']))
    .assign(key=lambda d: d.apply(lambda s: f"{s['Lot Number']}|{s['Inspection Date'].date()}|{s['Inspector']}|{s['Item Number']}" 
                                   if s['Path'] != 'QIM' 
                                   else s['ID'], axis=1))
    .query('PIC != "Missing in ETQ"')
    .groupby(['key', 'PIC'], as_index=False)['Qty EA'].sum()
)

# 直接构建dfs_by_type，避免使用eval或locals
criteria_types = ['Functional', 'Dimensional', 'Visual']
dfs_by_type = {
    f"normal_{criteria}": get_criteria_df(df_normal, criteria) 
    for criteria in criteria_types
}
dfs_by_type.update({
    f"reduce_{criteria}": get_criteria_df(df_reduce, criteria) 
    for criteria in criteria_types
})

# 使用函数创建每个类型的样本大小DataFrame
sample_size_dfs = [create_sample_size_df(df_base, dfs_by_type[key], key) for key in dfs_by_type.keys()]

# 合并所有样本大小DataFrame
final_df = reduce(lambda left, right: pd.merge(left, right, on='key', how='left'), [df_base] + sample_size_dfs)


In [148]:
df_gy = (
    final_df.groupby('PIC',as_index=False)[[
        'normal_Functional Sample Size', 'normal_Dimensional Sample Size', 'normal_Visual Sample Size',
        'reduce_Functional Sample Size', 'reduce_Dimensional Sample Size', 'reduce_Visual Sample Size'
    ]]
    .sum()
    .rename(columns=lambda x: x.replace(' Sample Size', ''))
)

In [145]:
(
    df_gy
    .assign(Functional_decreasing_qty = lambda d : d.normal_Functional - d.reduce_Functional,
            Dimensional_decreasing_qty = lambda d : d.normal_Dimensional - d.reduce_Dimensional,
            Visual_decreasing_qty = lambda d : d.normal_Visual - d.reduce_Visual,
            Functional_decreasing_per = lambda d : d.reduce_Functional / d.normal_Functional - 1,
            Dimensional_decreasing_per = lambda d : d.reduce_Dimensional /  d.normal_Dimensional - 1,
            Visual_decreasing_per = lambda d : d.reduce_Visual / d.normal_Visual - 1
            )
    .fillna(0)
).to_excel('final.xlsx')

,PIC,normal_Functional,normal_Dimensional,normal_Visual,reduce_Functional,reduce_Dimensional,reduce_Visual,Functional_decreasing_qty,Dimensional_decreasing_qty,Visual_decreasing_qty,Functional_decreasing_per,Dimensional_decreasing_per,Visual_decreasing_per
0,PIC-00001,2119.0,0.0,19910.0,815.0,0.0,7961.0,1304.0,0.0,11949.0,-0.615385,-0.615385,-0.615385
1,PIC-00002,39.0,39.0,240.0,15.0,15.0,96.0,24.0,24.0,144.0,-0.615385,-0.615385,-0.615385
2,PIC-00004,65.0,0.0,490.0,25.0,0.0,196.0,40.0,0.0,294.0,-0.615385,-0.615385,-0.615385
3,PIC-00005,16180.0,0.0,16270.0,6459.0,0.0,6495.0,9721.0,0.0,9775.0,-0.600803,-0.600803,-0.600803
4,PIC-00013,0.0,0.0,100.0,0.0,0.0,40.0,0.0,0.0,60.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Qual-96703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
170,Qual-98465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
171,Qual-98611,0.0,5.0,370.0,0.0,5.0,148.0,0.0,0.0,222.0,0.000000,0.000000,0.000000
172,Qual-99085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
